### Numerical investigation of the nullity of the $\Delta{\chi}$ operator

Let consider a coarse and a fine discretization for a block in the GFM framework, and the transmission condition for both fine and coarse level :

$$\phi {\bf u}_{n+1}= \chi{\bf u}_{n}$$
$$\tilde{\phi}{\bf u}_{n+1}= \tilde{\chi}{\bf u}_{n}$$

with $\chi$, $\tilde{\chi}$ the transmission operators and $\phi$, $\tilde{\phi}$ the integration operators.
Now we have $T_F^C$ and $T_C^F$ respectively the restriction and prolongation operator between coarse and fine grid,
and from that we define

$$\Delta{\chi} = T_F^C\chi - \tilde{\chi}T_C^F$$

Many block iterations are simplified under the condition that $\Delta{\chi}=0$, here is a numerical investigation of the situations when this nullity occurs.

In [1]:
# Script import setup
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from gfm.base import GFMSolver

To simplify, we consider the following settings for the Dahlquist problem :

In [2]:
params = {'lam': 1+1j, 'u0': 1, 'dt': 0.1*np.pi, 'L':10}

And here we give the list of discretization and numerical method that are investigated :

In [3]:
methods = ['COLLOCATION', 'BE', 'FE', 'TRAP', 'RK2', 'RK4', 'MULTISTEP']
forms = ['Z2N', 'N2N']

distr = ['EQUID', 'LEGENDRE', 'CHEBY-1', 'CHEBY-2']
qTypes = ['LOBATTO', 'RADAU-I', 'RADAU-II', 'GAUSS']

- **methods** : either we consider one step of a collocation method, or different time steps of a Runge-Kutta or multistep method.
- **forms** : either zero-to-node or node-to-node formulation
- **distr** : distribution of points used for the block discretization. Can be equidistant points (EQUID), or a distribution associated to orthogonal polynomials with respect to a given weight function (ex: LEGENDRE for weight function equal to one).
- **qTypes** : the quadrature type used with a given distribution. While GAUSS consider the standard node distribution for numerical Gauss quadrature (no inclusion of the boundary points), LOBATTO includes the two boundary points, RADAU-I and RADAU-II include respectively the left and right boundary point only.

Now any combination of those four parameters can be considered. Below, we build a test function that take one value for each four parameter, and build the $\Delta\chi$ associated to this block discretization. Number of points for coarse and fine level are chosen randomly in $\{2,...,10\}$.

In [4]:
def runTest(method, form, distr, qType):
    s = GFMSolver(**params)
    # Randomly chose number of fine and coarse nodes
    M = np.random.randint(3, 11)
    MCoarse = np.random.randint(2, M)
    # Build fine level
    lvlParams = {
        'M': M,
        'method': method,
        'nodes': distr,
        'qType': qType,
        'form': form
    }
    s.setFineLevel(**lvlParams)
    # Build coarse level
    s.setCoarseLevel(MCoarse, nodes=distr, qType=qType)
    # Scaled norm of deltaChi
    norm = np.linalg.norm(s.deltaChi, ord=np.inf)/(M/MCoarse)
    # Check if coarse nodes are subset of fine nodes
    subset = bool(np.prod([np.isin(node, s.nodes) for node in s.nodesCoarse]))
    return  norm < 1e-14, subset, norm, M, MCoarse, s.nodes, s.nodesCoarse

In [5]:
runTest('COLLOCATION', 'Z2N', 'LEGENDRE', 'GAUSS')

(False,
 False,
 1.7137385823740547,
 9,
 3,
 array([0.01591988, 0.08198445, 0.19331428, 0.33787329, 0.5       ,
        0.66212671, 0.80668572, 0.91801555, 0.98408012]),
 array([0.11270167, 0.5       , 0.88729833]))

The function returns 7 variables :

1. A boolean indicating if the scaled $L^\infty$ norm of $\Delta\chi$ is below 1e-14
2. A boolean indicating if the coarse nodes are a subset of the fine nodes
3. The scaled $L^\infty$ norm of $\Delta\chi$
4. M : the number of fine points
5. MCoarse : the number of coarse points
6. The fine points
7. The coarse points

Now we implement a function that run 50 tests (with random number of points) and returns, for each configuration, the following strings :

- **yes** : $\Delta\chi=0$ is always verified
- **subset** : $\Delta\chi=0$ only when the coarse points are a subset of the fine nodes
- **no** : neither 'yes' or 'subset'

In [6]:
def checkConfig(method, form, distr, qType):
    results = np.array([runTest(method, form, distr, qType)[:2] for i in range(50)])
    if np.all(results[:,0]):
        return 'yes'
    elif np.any(results[:,0]) and (np.all(results[:,0] == results[:,1])):
        return 'subset'
    else:
        return 'no'

Now we create a multi-indexed pandas dataframe with methods/form in columns, and distribution/quadrature type in row. 

In [7]:
idxCol = pd.MultiIndex.from_product([methods, forms])
idxRow = pd.MultiIndex.from_product([distr, qTypes])
table = pd.DataFrame(None, index=idxRow, columns=idxCol)

And finally, we run the checkConfig function for each configuration and prompt the results stored in the dataframe :

In [8]:
for method, form in table.columns:
    for distr, qType in table.index:
        table.loc[(distr, qType), (method, form)] = checkConfig(method, form, distr, qType)
table

COLLOCATION           BE           FE         TRAP          \
                          Z2N     N2N  Z2N     N2N  Z2N     N2N  Z2N     N2N   
EQUID    LOBATTO          yes  subset  yes  subset  yes  subset  yes  subset   
         RADAU-I           no      no   no      no   no      no   no      no   
         RADAU-II         yes      no  yes      no  yes      no  yes      no   
         GAUSS             no      no   no      no   no      no   no      no   
LEGENDRE LOBATTO          yes  subset  yes  subset  yes  subset  yes  subset   
         RADAU-I           no      no   no      no   no      no   no      no   
         RADAU-II         yes      no  yes      no  yes      no  yes      no   
         GAUSS             no      no   no      no   no      no   no      no   
CHEBY-1  LOBATTO          yes  subset  yes  subset  yes  subset  yes  subset   
         RADAU-I           no      no   no      no   no      no   no      no   
         RADAU-II         yes      no  yes      no  yes      no  yes      no   
         GAUSS             no      no   no      no   no      no   no      no   
CHEBY-2  LOBATTO          yes  subset  yes  subset  yes  subset  yes  subset   
         RADAU-I           no      no   no      no   no      no   no      no   
         RADAU-II         yes      no  yes      no  yes      no  yes      no   
         GAUSS             no      no   no      no   no      no   no      no   

                   RK2          RK4         MULTISTEP      
                   Z2N     N2N  Z2N     N2N       Z2N N2N  
EQUID    LOBATTO   yes  subset  yes  subset        no  no  
         RADAU-I    no      no   no      no        no  no  
         RADAU-II  yes      no  yes      no        no  no  
         GAUSS      no      no   no      no        no  no  
LEGENDRE LOBATTO   yes  subset  yes  subset        no  no  
         RADAU-I    no      no   no      no        no  no  
         RADAU-II  yes      no  yes      no        no  no  
         GAUSS      no      no   no      no        no  no  
CHEBY-1  LOBATTO   yes  subset  yes  subset        no  no  
         RADAU-I    no      no   no      no        no  no  
         RADAU-II  yes      no  yes      no        no  no  
         GAUSS      no      no   no      no        no  no  
CHEBY-2  LOBATTO   yes  subset  yes  subset        no  no  
         RADAU-I    no      no   no      no        no  no  
         RADAU-II  yes      no  yes      no        no  no  
         GAUSS      no      no   no      no        no  no

We can make a few observations :

1. Multistep methods (actually, 2nd order Adams-Bashforth) never satisfy $\Delta\chi=0$.
2. Distributions using RADAU-I or GAUSS quadrature type never satisfy $\Delta\chi=0$.
3. When RADAU-II quadrature type is used, $\Delta\chi=0$ only if the zero-to-nodes formulation is used.
4. When LOBATTO quadrature type is used, $\Delta\chi=0$ is always satisfied for zero-to-nodes formulation, and for the node-to-node formulation only if the coarse points are a subset of the fine points.

Now we define the following function to investigate a little bit more the impact of $\Delta\chi$ by looking at its norm :

In [9]:
def normDeltaChi(method, form, distr, qType):
    return np.mean([runTest(method, form, distr, qType)[2] for i in range(50)])

In [10]:
normDeltaChi('COLLOCATION', 'Z2N', 'EQUID', 'GAUSS')

175.55375558443956

Similarly as before, we present all the configuration results using a pandas dataframe :

In [11]:
for method, form in table.columns:
    for distr, qType in table.index:
        table.loc[(distr, qType), (method, form)] = normDeltaChi(method, form, distr, qType)
pd.set_option('display.float_format', lambda x: f'{x:.3g}')
table

COLLOCATION               BE              FE          \
                          Z2N     N2N      Z2N    N2N      Z2N     N2N   
EQUID    LOBATTO     6.97e-17 0.00724 4.89e-17 0.0064 4.71e-17 0.00835   
         RADAU-I          221     187     2.56   2.42     2.61    3.12   
         RADAU-II     7.8e-17   0.418 1.05e-16  0.419 1.24e-16   0.408   
         GAUSS            224    60.7     3.15   1.94     2.45    2.17   
LEGENDRE LOBATTO     4.42e-17  0.0332 5.68e-17 0.0438 3.99e-17  0.0365   
         RADAU-I         3.98    3.56     1.14   1.05     1.07    1.08   
         RADAU-II    1.22e-16   0.328 1.34e-16   0.32 1.09e-16   0.322   
         GAUSS           2.79     2.1    0.891  0.605    0.887   0.651   
CHEBY-1  LOBATTO     5.96e-17  0.0329 6.31e-17 0.0301 7.32e-17  0.0383   
         RADAU-I          3.6    3.69     1.11   1.13     1.15   0.974   
         RADAU-II    1.01e-16   0.293 9.84e-17  0.333 1.11e-16   0.315   
         GAUSS           2.85    1.93    0.879  0.641     0.95   0.635   
CHEBY-2  LOBATTO     5.62e-17  0.0343  5.2e-17 0.0466 3.96e-17  0.0274   
         RADAU-I         3.84    3.39     1.08   1.12     1.07    1.15   
         RADAU-II    1.32e-16   0.308 1.18e-16  0.323 1.24e-16   0.309   
         GAUSS           2.84    2.08    0.908  0.552    0.873   0.646   

                      TRAP              RK2              RK4          \
                       Z2N     N2N      Z2N     N2N      Z2N     N2N   
EQUID    LOBATTO  3.88e-17 0.00543 4.02e-17 0.00797  4.1e-17 0.00923   
         RADAU-I      2.37    1.88     2.72    2.79     1.87    2.16   
         RADAU-II 1.07e-16   0.415 8.64e-17   0.408 1.25e-16   0.409   
         GAUSS        2.15    2.21     2.36    1.91     2.24    2.65   
LEGENDRE LOBATTO  5.02e-17  0.0378 6.27e-17  0.0375 5.37e-17  0.0402   
         RADAU-I      1.04    1.04     1.02    1.17     1.12    1.08   
         RADAU-II  1.1e-16   0.314 1.16e-16   0.316 1.09e-16   0.328   
         GAUSS       0.879   0.605    0.901   0.572    0.872   0.577   
CHEBY-1  LOBATTO  6.43e-17  0.0397 6.91e-17  0.0304  4.7e-17  0.0327   
         RADAU-I      1.05    1.12     1.14    1.07      1.1    1.15   
         RADAU-II 1.04e-16     0.3 9.73e-17   0.298 1.21e-16   0.289   
         GAUSS       0.809   0.573    0.818   0.626    0.847   0.619   
CHEBY-2  LOBATTO  5.13e-17  0.0414 5.99e-17  0.0344  5.4e-17   0.034   
         RADAU-I       1.1    1.07     1.17    1.09     1.15    1.09   
         RADAU-II 1.35e-16   0.342 9.32e-17   0.326 1.18e-16    0.31   
         GAUSS       0.937   0.557    0.864   0.618    0.832   0.559   

                  MULTISTEP         
                        Z2N    N2N  
EQUID    LOBATTO     0.0334 0.0473  
         RADAU-I       2.41   1.89  
         RADAU-II     0.106  0.562  
         GAUSS         2.28   2.31  
LEGENDRE LOBATTO     0.0415 0.0708  
         RADAU-I       1.14   1.03  
         RADAU-II    0.0804  0.376  
         GAUSS        0.935  0.658  
CHEBY-1  LOBATTO     0.0364  0.075  
         RADAU-I        1.1   1.21  
         RADAU-II     0.068  0.399  
         GAUSS        0.939  0.698  
CHEBY-2  LOBATTO     0.0358 0.0805  
         RADAU-I          1      1  
         RADAU-II    0.0668  0.379  
         GAUSS         0.92  0.617

In particular, we can observe that some configurations induce larger norm for the $\Delta\chi$ operator than other, for instance the equidistant points with RADAU-I and GAUSS, or the collocation method with RADAU-I and GAUSS nodes, compared with Runge-Kutta or Multistep.

Now we focus on collocation with Legendre point distribution, using the zero-to-node formulation, and investigate when we use different quadrature types for coarse and fine points :

In [12]:
def runTest2(qTypeFine, qTypeCoarse):
    s = GFMSolver(**params)
    # Randomly chose number of fine and coarse nodes
    M = np.random.randint(3, 11)
    MCoarse = np.random.randint(2, M)
    # Build fine level
    lvlParams = {
        'M': M,
        'method': 'COLLOCATION',
        'nodes': 'LEGENDRE',
        'qType': qTypeFine,
        'form': 'Z2N'
    }
    s.setFineLevel(**lvlParams)
    # Build coarse level
    s.setCoarseLevel(MCoarse, nodes='LEGENDRE', qType=qTypeCoarse)
    # Scaled norm of deltaChi
    return np.linalg.norm(s.deltaChi, ord=np.inf)/(M/MCoarse), M, MCoarse, s.nodes, s.nodesCoarse

In [13]:
runTest2('GAUSS', 'LOBATTO')

(1.2688263138573217e-16,
 7,
 2,
 array([0.02544604, 0.12923441, 0.29707742, 0.5       , 0.70292258,
        0.87076559, 0.97455396]),
 array([0., 1.]))

In [14]:
def normDeltaChi2(qTypeFine, qTypeCoarse):
    return np.mean([runTest2(qTypeFine, qTypeCoarse)[0] for i in range(100)])

In [15]:
normDeltaChi2('GAUSS', 'LOBATTO')

4.577125715480671e-16

In [16]:
table = pd.DataFrame(None, index=qTypes, columns=qTypes)
table.index.name='Fine'
table.columns.name='Coarse'

Again, we present all the results in a pandas dataframe :

In [17]:
for qTypeCoarse in table.columns:
    for qTypeFine in table.index:
        table.loc[qTypeFine, qTypeCoarse] = normDeltaChi2(qTypeFine, qTypeCoarse)
pd.set_option('display.float_format', lambda x: f'{x:.3g}')
table

Coarse,LOBATTO,RADAU-I,RADAU-II,GAUSS
Fine,,,,
LOBATTO,7.17e-17,2.66,8.68e-17,1.89
RADAU-I,3.64e-16,3.54,4.39e-16,3.24
RADAU-II,1.17e-16,3.2,1.11e-16,1.99
GAUSS,4e-16,3.6,3.27e-16,2.74


Here we can see that the nullity of $\Delta\chi$ depends only on which quadrature type is used on the coarse level, and does not depend on the fine level discretization.